# Introduction to Julia

#### John Pearson
#### StatML Group Meeting
#### October 8, 2015

# What is Julia?

- new programming language (public since 2012)
- brought to you by MIT, Julia Computing
- focused on scientific computing applications
- aims to be a high-level prototyping language that is also fast enough for serious simulation
    - as easy to learn and flexible as Python or R
    - within 2x the speed of C, C++, or FORTRAN
- Currently (almost) 0.4, with very active developer community (more later)


# Reasons to Consider Julia

- you mostly write your own algorithms
- you need to write code that is very fast
- you’d like to prototype and optimize within the same language
- you want zero boilerplate calls to C, Python (and soon C++)
- you need powerful metaprogramming/macros
- you want access to state-of-the-art automatic differentiation and optimization
- you want to use Greek letters in variable names
- you are interested in programming language design
- you want to get in on the ground floor


# Reasons Julia may not be for you (yet)

- you mostly use packages written by others
- you rely critically on several packages that aren't available
- you don’t have much need to optimize your code
- you perform mostly data analysis and model building, as opposed to algorithm development
- you prioritize stability and reliability 
    - (or don’t write a lot of tests for your own code)
- you don’t want to devote time to keeping up with changes
- you have zero background with compiled languages

# What makes Julia different?

- just in time (JIT) compilation
- rich type system
- multiple dispatch
- metaprogramming (programs that write programs: think C macros, STAN)
- no more fear of writing loops

More on each of these later!

# Learning Julia

- [Julia homepage](http://julialang.org/)
- [Learning resources](http://julialang.org/learning/)
- [Docs](http://julia.readthedocs.org/en/release-0.4/) 
    - more comprehensive and up-to-date than most other info
- [YouTube](https://www.youtube.com/user/JuliaLanguage)
- [GitHub](https://github.com/JuliaLang/julia)
    - this is where most of the development discussion takes place
    - being GitHub-friendly currently important
- [julia-users list](https://groups.google.com/forum/#!forum/julia-users)

# Just-in-time (JIT) compilation

- compilers transform written code to machine code
    - much faster
    - typically a separate step
- JIT compilers work at runtime
    - can be very fast with modern methods
- Julia uses [LLVM](http://llvm.org/) as a backend

# [The Julia compilation process](http://blog.leahhanson.us/julia-introspects.html)

- Parsing (Abstract Syntax Tree; AST)
    - macro expansion
- Code lowering
- Code typing
- LLVM
- machine code



# Julia's type system

- variables in code have types (int, float, etc.)
- compilers use type information to generate efficient code
- like objects, but focus on data, not behavior
    - [Execution in the Kingdom of Nouns](http://steve-yegge.blogspot.com/2006/03/execution-in-kingdom-of-nouns.html)
- key to understanding how Julia code is organized and what makes it fast

# More on types

- two types of types:
    - Abstract: 
        - never instantiated
        - tree structure (no multiple inheritance)
    - Concrete:
        - only "leaf types"
        - fast, optimized code
- key functions:
    - `typeof`
    - `super` and `subtypes`
    - `::` and `<:`

In [16]:
typeof(1)

Int64

In [2]:
typeof(1.1)

Float64

Strings always have " ", chars have ' '

In [8]:
typeof('a')

Char

In [9]:
typeof("alphabet")

ASCIIString

We can find out about the type tree:

In [10]:
super(Float64)

AbstractFloat

In [11]:
super(AbstractFloat)

Real

In [12]:
super(Real)

Number

In [13]:
super(Number)

Any

In [14]:
super(Any)

Any

More on the type tree:

Everything is a subtype of `Any`:

In [15]:
subtypes(Any)

241-element Array{Any,1}:
 AbstractArray{T,N}                        
 AbstractChannel{T}                        
 AbstractRNG                               
 AbstractString                            
 Any                                       
 Associative{K,V}                          
 Base.AbstractCmd                          
 Base.AbstractMsg                          
 Base.AbstractZipIterator                  
 Base.Cartesian.LReplace{S<:AbstractString}
 Base.Combinations{T}                      
 Base.Count{S<:Number}                     
 Base.Cycle{I}                             
 ⋮                                         
 TypeVar                                   
 Type{T}                                   
 UniformScaling{T<:Number}                 
 Val{T}                                    
 Vararg{T}                                 
 VersionNumber                             
 Void                                      
 WeakRef                                   
 Worke

We can also test type relationships

In [17]:
isa(1.1, Float64)

true

In [18]:
Number <: Any

true

In [19]:
Real <: Number

true

In [20]:
Integer <: Real

true

In [21]:
Integer <: AbstractFloat

false

Finally, some types can have parameters (like template objects in C++/Java):

In [22]:
aa = [1:10]

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [23]:
typeof(aa)

Array{Int64,1}

In [24]:
typeof(rand(5, 5))

Array{Float64,2}

# Multiple Dispatch

If Julia's types are just data, not objects, how do we encode type behavior?

Julia (like R, C++ operators), choosing among *different* functions of the *same name* based on the types of arguments. This is called **multiple dispatch**.

In [25]:
?imag

search: 

```
imag(z)
```

Return the imaginary part of the complex number `z`


imag UniformScaling ismatch ismarked minmax eigmax BitMatrix indmax



In [26]:
methods(imag)

# 15 methods for generic function "imag":
imag(z::Complex{T<:Real}) at complex.jl:32
imag(x::Real) at complex.jl:34
imag{T<:Real}(x::Union{DenseArray{T<:Real,N},SubArray{T<:Real,N,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at arraymath.jl:29
imag(A::Union{DenseArray{T,N},SubArray{T,N,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at arraymath.jl:27
imag(M::SymTridiagonal{T}) at linalg/tridiag.jl:63
imag(M::Tridiagonal{T}) at linalg/tridiag.jl:320
imag(A::UpperTriangular{T,S<:AbstractArray{T,2}}) at linalg/triangular.jl:46
imag(A::LowerTriangular{T,S<:AbstractArray{T,2}}) at linalg/triangular.jl:47
imag(A::Base.LinAlg.UnitLowerTriangular{T,S<:AbstractArray{T,2}}) at linalg/triangular.jl:48
imag(A::Base.LinAlg.UnitUpperTriangular{T,S<:AbstractArray{T,2}}) at linalg/triangular.jl:49
imag(D::Diagonal{T}) at linalg/diagonal.jl:60
imag(M::Bidiagonal{T}) at linalg/bidiag.jl:113
imag{Tv<:Complex{T<:Real},Ti}(A::SparseMatrixCSC{Tv<:Complex{T<:Real},Ti}) at sparse/sparsematrix.jl:548
imag{Tv<:Number,Ti}(A::SparseMatrixCSC{Tv<:Number,Ti}) at sparse/sparsematrix.jl:552
imag{T<:Real}(x::AbstractArray{T<:Real,N}) at abstractarraymath.jl:47

So, for instance:

In [27]:
function foo(x::AbstractFloat)
    x^2
end

function foo(x::Integer)
    x/2
end

foo (generic function with 2 methods)

In [28]:
foo(2.0)

4.0

In [29]:
foo(2)

1.0

# But surely this is madness!

How do we avoid combinatorial explosion in definitions for functions that take multiple arguments?

Julia has two complementary means of dealing with this:
- conversion: define a `convert` method telling Julia how to translate one data type to another
- promotion: define `promote_rule` and `promote_type`, telling Julia how to treat two *types* on an equal footing

Most conversions happen the way you would expect:

In [30]:
convert(Float64, 1)

1.0

In [32]:
convert(Complex, 1.5)

1.5 + 0.0im

In [33]:
promote(1, 2.5)

(1.0,2.5)

In [34]:
promote(1, 0.5im)

(1.0 + 0.0im,0.0 + 0.5im)

But we can write our own conversion rules:

In [36]:
type Lefty
    x::Number
end

type Righty
    x::Number
end

In [39]:
aa = Righty(5)

Righty(5)

In [38]:
import Base.convert
convert(::Type{Lefty}, r::Righty) = Lefty(-r.x)

convert (generic function with 536 methods)

In [40]:
convert(Lefty, aa)

Lefty(-5)

Promotion works by describing what types of "least common denonimators" are possible between types.

In the case of a Rational type for rational numbers, we might define

In [41]:
promote_rule{T<:Integer}(::Type{Rational{T}}, ::Type{T}) = Rational{T}
promote_rule{T<:Integer,S<:Integer}(::Type{Rational{T}}, ::Type{S}) = Rational{promote_type(T,S)}

promote_rule (generic function with 2 methods)

Rough guide: `promote` is about conversion *strategy*, while `convert` is about *implementation*